In [4]:
import pandas as pd
import geopandas as gpd
import folium
from branca.colormap import linear

# Step 1: 데이터 불러오기
jobs_path = "../data/cleaned_jobs.csv"  # 파일 경로
jobs_df = pd.read_csv(jobs_path)

# Step 2: 근무지역에서 ~구 추출
def extract_district(location):
    if pd.notna(location):
        for part in location.split():
            if "구" in part:
                return part
    return None

jobs_df['구'] = jobs_df['근무지역'].apply(extract_district)

# Step 3: 데이터 그룹화 (~구별 일자리 수 계산)
job_counts = jobs_df.groupby('구').size().reset_index(name='일자리수')

# Step 4: GeoJSON 데이터 읽기
boundary_path = "../data/서울_자치구_경계_2017.geojson"  # 서울 자치구 경계 데이터 경로
geo_data = gpd.read_file(boundary_path)

# 자치구 이름 정리
geo_data['SIG_KOR_NM'] = geo_data['SIG_KOR_NM'].str.strip()

# Step 5: GeoJSON 데이터와 일자리 데이터 병합
merged = geo_data.merge(job_counts, left_on='SIG_KOR_NM', right_on='구', how='left')
merged['일자리수'] = merged['일자리수'].fillna(0)  # 결측값 처리

# Step 6: 지도 생성 및 시각화
m = folium.Map(location=[37.5665, 126.9780], zoom_start=11)

# 컬러맵 생성
colormap = linear.YlOrRd_09.scale(merged['일자리수'].min(), merged['일자리수'].max())
colormap.caption = '근무지역별 일자리 밀도'

# 자치구별 밀도 시각화
folium.Choropleth(
    geo_data=merged,
    data=merged,
    columns=['name', '일자리수'],
    key_on='feature.properties.SIG_KOR_NM',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='근무지역별 일자리수'
).add_to(m)

# 지도에 컬러맵 추가
colormap.add_to(m)

# 결과 저장
output_path = "seoul_jobs_density_by_district.html"
m.save(output_path)
print(f"지도가 {output_path}에 저장되었습니다.")


KeyError: "None of ['name'] are in the columns"

In [6]:
# 지도 생성 및 시각화
m = folium.Map(location=[37.5665, 126.9780], zoom_start=11)

# 컬러맵 생성
colormap = linear.YlOrRd_09.scale(merged['일자리수'].min(), merged['일자리수'].max())
colormap.caption = '근무지역별 일자리 밀도'

# 자치구별 밀도 시각화
folium.Choropleth(
    geo_data=merged,  # 병합된 GeoJSON 데이터
    data=merged,  # 데이터프레임
    columns=['SIG_KOR_NM', '일자리수'],  # 지도에서 사용할 컬럼
    key_on='feature.properties.SIG_KOR_NM',  # GeoJSON 데이터의 키
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='근무지역별 일자리수'
).add_to(m)

# 지도에 컬러맵 추가
colormap.add_to(m)

# 결과 저장
output_path = "seoul_jobs_density_by_district.html"
m.save(output_path)
print(f"지도가 {output_path}에 저장되었습니다.")


지도가 seoul_jobs_density_by_district.html에 저장되었습니다.
